# **Live Code 3**
---
Nadia Oktiarsy, FSDS Batch-016 RMT (Buddy: Hana)

**Problem Statements**

To evaluate number of crime in Austin TX, year 2016.
- Where is the highest number of criminal cases? Which cases is the highest cases regardless the district?

**Penjabaran Masalah**
1. Tampilkan tabel crime sampai 5 baris pertama
2. Apa saja jenis tindak kejahatan yang terjadi di kota Austin selama tahun 2016?
3. Berapa banyak kasus tindak kejahatan berdasarkan kasus pembersihannya (clearance status) selama tahun 2016?
4. Jenis kejahatan apa saja yang paling banyak belum terselesaikan kasusnya di tahun 2016?
5. Di bulan apa di tahun 2016 kasus pencurian (theft) sangat banyak terjadi?
6. Di distrik apa yang paling banyak terjadi kasus pencurian selama 2016?
7. Berapa lama rata-rata waktu (dalam hari) kasus tindak kejahatan 'theft' terselesaikan sejak kasus tersebut terangkat selama tahun 2016? (Hint: Hitung selisih hari dari timestamp hingga clearance date)

In [1]:
# Importing libraries
from google.cloud import bigquery
import numpy as np
import pandas as pd

from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
# Loading Project from Bigquery
project_id = "hacktiv8-016-nadiaoktiarsy"
client = bigquery.Client(project=project_id)

**1. First 5 rows crime table**

In [3]:
client.query('''
SELECT *
FROM `bigquery-public-data.austin_crime.crime`
LIMIT 5
''').to_dataframe()

,unique_key,address,census_tract,clearance_date,clearance_status,council_district_code,description,district,latitude,longitude,location,location_description,primary_type,timestamp,x_coordinate,y_coordinate,year,zipcode
0,20165002898,"Austin, TX",None,2016-03-14 12:00:00+00:00,Not cleared,None,RAPE,UK,None,None,None,nan,Rape,2016-01-19 12:00:00+00:00,None,None,2016,None
1,20165004843,"Austin, TX",None,2016-03-31 12:00:00+00:00,Not cleared,None,RAPE,UK,None,None,None,nan,Rape,2016-01-25 12:00:00+00:00,None,None,2016,None
2,2016250988,"Austin, TX",None,2016-09-16 12:00:00+00:00,Not cleared,None,RAPE,UK,None,None,None,nan,Rape,2016-01-25 12:00:00+00:00,None,None,2016,None
3,20165003839,"Austin, TX",None,2016-02-09 12:00:00+00:00,Not cleared,None,AGG RAPE OF A CHILD,UK,None,None,None,nan,Rape,2016-01-27 12:00:00+00:00,None,None,2016,None
4,2016321746,"Austin, TX",None,2016-03-22 12:00:00+00:00,Not cleared,None,RAPE OF A CHILD,UK,None,None,None,nan,Rape,2016-02-01 12:00:00+00:00,None,None,2016,None


**2. Crime categories in Austin, TX**

There are 7 categories of crime in Austin TX, arranged by name. By using SELECT DISTINCT, we can find the unique categories of crime. Definition of the catogories:

Category | Definition
---|---
Agg Assault | aggravated assault, any intentional act that causes another person to fear an attack or imminent physical harm
Auto Theft | the criminal act of stealing or attempting to steal a motor vehicle
Burglary | breaking and entering and sometimes housebreaking
Murder | the unlawful killing of another human without justification or valid excuse
Rape | Number of people died from COVID-19
Robbery | the crime of taking or attempting to take anything of value by force, threat of force, or by use of fear
Theft | the act of taking another person's property or services without that person's permission or consent with the intent to deprive the rightful owner of it.

The table result as below:


In [141]:
# Crime categories
client.query('''
SELECT DISTINCT
  primary_type as crime_category,
FROM `bigquery-public-data.austin_crime.crime`
WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
ORDER BY primary_type ASC;
''').to_dataframe()

,crime_category
0,Agg Assault
1,Auto Theft
2,Burglary
3,Murder
4,Rape
5,Robbery
6,Theft


**3. Number of Criminal Cases in 2016**

Based on the observation of the data, there are many "Not Cleared" status cases which should be noticed, compared with cleared_by_arrest and cleard_by_exception. This means, the work of clearing criminal cases should be evaluated more, especially to understand why there are many "not cleared" cases.

In [148]:
client.query('''
SELECT DISTINCT
   EXTRACT(year FROM clearance_date) AS year,
   EXTRACT(month FROM clearance_date) AS month,
   COUNT(clearance_status) AS clearance_status,
   COUNT(CASE WHEN clearance_status='Cleared by Arrest' THEN clearance_status ELSE NULL END) AS cleared_by_arrest,
   COUNT(CASE WHEN clearance_status='Cleared by Exception' THEN clearance_status ELSE NULL END) AS cleared_by_exception,
   COUNT(CASE WHEN clearance_status='Not cleared' THEN clearance_status ELSE NULL END) AS not_cleared,
FROM `bigquery-public-data.austin_crime.crime` 
WHERE clearance_status IS NOT NULL AND
      clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
GROUP BY year, month
ORDER BY year, month, clearance_status
''').to_dataframe()

,year,month,clearance_status,cleared_by_arrest,cleared_by_exception,not_cleared
0,2016,1,3107,393,95,2619
1,2016,2,2858,460,75,2323
2,2016,3,2996,408,75,2513
3,2016,4,2867,439,79,2349
4,2016,5,3132,467,85,2580
5,2016,6,2895,508,68,2319
6,2016,7,2639,392,74,2173
7,2016,8,3266,454,72,2740
8,2016,9,2818,424,89,2305
9,2016,10,2849,377,84,2388


**4. The highest type of cases with Not Cleared status in 2016**

**Theft** has the highest number of cases with not cleared status in 2016 with the result of `2111`.

In [147]:
client.query('''

SELECT primary_type, count_not_cleared, Rank
FROM
    (SELECT DISTINCT
      primary_type,
      count_not_cleared,
      RANK() OVER(PARTITION BY primary_type ORDER BY count_not_cleared DESC) AS Rank
    FROM (SELECT DISTINCT
            primary_type, 
            EXTRACT(year FROM clearance_date) AS year,
            EXTRACT(month FROM clearance_date) AS month, 
            COUNT(CASE WHEN clearance_status='Not cleared' THEN clearance_status ELSE NULL END) AS count_not_cleared,  
          FROM `bigquery-public-data.austin_crime.crime` 
          WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
          GROUP BY year, month, primary_type) AS not_cleared) AS table_not_cleared
WHERE Rank = 1
ORDER BY count_not_cleared DESC;

''').to_dataframe()

,primary_type,count_not_cleared,Rank
0,Theft,2111,1
1,Burglary,432,1
2,Auto Theft,147,1
3,Agg Assault,93,1
4,Robbery,79,1
5,Rape,53,1
6,Murder,0,1


**5. Month in 2016 with the highest number of theft**

Theft cases happened mostly in August 2016, with number of cases: `2375`

In [72]:
client.query('''

SELECT year, month, primary_type, count_cases, Rank
FROM
    (SELECT DISTINCT
      year,
      month, primary_type,
      count_cases,
      RANK() OVER(PARTITION BY primary_type ORDER BY count_cases DESC) AS Rank
    FROM (SELECT DISTINCT
            primary_type, 
            EXTRACT(year FROM clearance_date) AS year,
            EXTRACT(month FROM clearance_date) AS month, 
            COUNT(clearance_status) AS count_cases, 
          FROM `bigquery-public-data.austin_crime.crime` 
          WHERE 
            primary_type = "Theft" AND
            clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
          GROUP BY year, month, primary_type) AS count_all_cases) AS table_count_cases
WHERE Rank = 1
ORDER BY year, month, count_cases DESC;

''').to_dataframe()

,year,month,primary_type,count_cases,Rank
0,2016,8,Theft,2375,1


In [71]:
client.query('''

SELECT year, month, primary_type, count_cases, Rank
FROM
    (SELECT DISTINCT
      year,
      month, primary_type,
      count_cases,
      RANK() OVER(PARTITION BY primary_type ORDER BY count_cases DESC) AS Rank
    FROM (SELECT DISTINCT
            primary_type, 
            EXTRACT(year FROM clearance_date) AS year,
            EXTRACT(month FROM clearance_date) AS month, 
            COUNT(clearance_status) AS count_cases, 
          FROM `bigquery-public-data.austin_crime.crime` 
          WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
          GROUP BY year, month, primary_type) AS count_all_cases) AS table_count_cases
WHERE Rank = 1
ORDER BY year, month, count_cases DESC;

''').to_dataframe()

,year,month,primary_type,count_cases,Rank
0,2016,1,Burglary,481,1
1,2016,1,Auto Theft,189,1
2,2016,6,Agg Assault,211,1
3,2016,8,Theft,2375,1
4,2016,11,Rape,96,1
5,2016,12,Robbery,116,1
6,2016,12,Murder,5,1


**6. District with the highest "Pencurian" cases in 2016**

"Pencurian" definition: Burglary, Theft, Robbery

- The highest Burglary in 2016 is in `District E`.
- The highest Theft in 2016 is in `District D`.
- The highest Robbery in 2016 is in `District E`.

These district should be watch out more especially for the incoming crimes.

In [88]:
client.query('''
SELECT DISTINCT
  district,
  COUNT(CASE WHEN primary_type='Theft' THEN primary_type ELSE NULL END) AS count_theft,
  COUNT(CASE WHEN primary_type='Robbery' THEN primary_type ELSE NULL END) AS count_robbery,
  COUNT(CASE WHEN primary_type='Burglary' THEN primary_type ELSE NULL END) AS count_burglary,
FROM `bigquery-public-data.austin_crime.crime` 
WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
GROUP BY district
''').to_dataframe()

,district,count_theft,count_robbery,count_burglary
0,UK,35,2,4
1,I,2897,131,591
2,H,2411,136,638
3,A,2928,62,534
4,88,3,0,3
5,B,3421,44,519
6,F,2857,83,669
7,D,3838,71,692
8,E,2835,192,776
9,C,1609,83,478


In [94]:
client.query('''
SELECT DISTINCT
  district,
  COUNT(CASE WHEN primary_type='Theft' THEN primary_type ELSE NULL END) AS count_theft,
FROM `bigquery-public-data.austin_crime.crime` 
WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
GROUP BY district
ORDER BY count_theft DESC
LIMIT 1
''').to_dataframe()

,district,count_theft
0,D,3838


In [93]:
client.query('''
SELECT DISTINCT
  district,
  COUNT(CASE WHEN primary_type='Robbery' THEN primary_type ELSE NULL END) AS count_robbery,
FROM `bigquery-public-data.austin_crime.crime` 
WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
GROUP BY district
ORDER BY count_robbery DESC
LIMIT 1
''').to_dataframe()

,district,count_robbery
0,E,192


In [95]:
client.query('''
SELECT DISTINCT
  district,
  COUNT(CASE WHEN primary_type='Burglary' THEN primary_type ELSE NULL END) AS count_burglary,
FROM `bigquery-public-data.austin_crime.crime` 
WHERE clearance_date BETWEEN "2016-01-01" AND "2016-12-31"
GROUP BY district
ORDER BY count_burglary DESC
LIMIT 1
''').to_dataframe()

,district,count_burglary
0,E,776


**7. Average range of days that Theft cases cleared in 2016**

The average range of days to take care theft cases is around 3 months, or 120 days. This will be useful for scheduling, to see if the administration and law system have a specific regulation about 3 months solving cases. If the cases take more than 3 months, then the law and administration should take notice more if there is a delayed schedule or not.

In [136]:
day_different = client.query('''
SELECT
  EXTRACT(day from clearance_date) - EXTRACT(day from timestamp) as day_clear
FROM `bigquery-public-data.austin_crime.crime` 
WHERE year =2016 AND
      clearance_status IS NOT NULL
GROUP BY  day_clear
''').to_dataframe()
day_different

,day_clear
0,-5
1,6
2,-9
3,-18
4,21
...,...
56,26
57,25
58,28
59,29


In [140]:
client.query('''

SELECT
  year,
	primary_type,
	AVG(day_difference) AS average_day
FROM
	(SELECT
    year,
    primary_type,
    date_diff(clearance_date, timestamp, day) as day_difference,
	FROM `bigquery-public-data.austin_crime.crime`
	WHERE 	clearance_date IS NOT NULL AND
		primary_type = "Theft" AND
		year =  2016
	GROUP BY year, primary_type, day_difference)
GROUP BY year, primary_type
''').to_dataframe()

,year,primary_type,average_day
0,2016,Theft,121.842795


## **Overall Analysis**

There are 7 categories of crime in Austin TX:
- Agg Assault
- Auto Theft
- Burglary
- Murder
- Rape
- Robbery
- Theft

There are still many cases which not cleared. This should be evaluated more, for example, the law system, the admnistration system, etc.

With the average case category is `theft` which mostly happened in **district D** with results of `3838 cases`. This means, district D should be cautious with the number of crime especially theft. Meanwhile, **district E** has more cases for `robbery` and `burglary` although less than 1000 cases.

It should be noted that the average clearance days is around 120 days, which takes around 3 month, regardless where the district is.